Uses the method to fit a Log Pearson Type III distribution to the flood peaks detailed at:

https://streamflow.engr.oregonstate.edu/analysis/floodfreq/index.htm


In [1]:
import numpy as np
import pandas as pd
import datetime

In [18]:
input_data='./input_data/'
output_data='./output_data/'

In [19]:
yearly = pd.read_csv(output_data + 'ngonye_flow_yearly.csv')
yearly.head(3)

,WaterYear,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume,Flow_mean_pct_var,Flow_max_pct_var,Flow_min_pct_var,Volume_pct_var,Flow_mean_5yr_mvCoefVar,MeanQ3070,ExceedanceMean,ExceedanceMedian,ExceedanceMeanQ3070
0,1924,111.407480,562.828075,992.089057,3452.248382,3340.840902,31.286520,-9.530461,-4.579856,-48.229072,-9.592773,NaN,663.910873,0.566,0.577,0.490
1,1925,158.541897,494.681083,1111.873684,4501.386001,4342.844104,35.064049,1.392813,24.418307,-26.325763,1.322977,NaN,560.136645,0.435,0.729,0.740
2,1926,192.895237,562.828075,990.425137,3313.040641,3120.145404,31.234047,-9.682195,-8.427558,-10.361805,-9.744403,19.172583,683.462554,0.577,0.577,0.457


In [20]:
yearly=yearly.sort_values('Flow_max',ascending=False)
yearly['rank'] = np.arange(len(yearly))+1
n=len(yearly)
yearly.head(3)

,WaterYear,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume,Flow_mean_pct_var,Flow_max_pct_var,Flow_min_pct_var,Volume_pct_var,Flow_mean_5yr_mvCoefVar,MeanQ3070,ExceedanceMean,ExceedanceMedian,ExceedanceMeanQ3070,rank
33,1957,283.607969,803.689297,2333.913589,9912.101075,9628.493106,73.602299,112.831788,173.970469,31.792297,112.685197,46.269093,1004.311633,0.000,0.164,0.087,1
32,1956,290.521945,623.809012,1521.627983,9153.793636,8863.271691,47.986060,38.758695,153.010852,35.005214,38.663123,43.617624,663.367961,0.153,0.396,0.500,2
44,1968,281.322274,876.938912,2149.190815,8539.833006,8258.510732,67.776882,95.986743,136.040980,30.730138,95.851755,45.484240,989.791070,0.011,0.044,0.098,3


In [21]:
yearly['log_q']=np.log10(yearly['Flow_max'])
av_log_q=yearly['log_q'].mean()
yearly['log_q_m2']=(yearly['log_q']-av_log_q)**2
yearly['log_q_m3']=(yearly['log_q']-av_log_q)**3
yearly['ret']=(n+1)/yearly['rank']
yearly['exceed']=1/yearly['ret']

yearly.head(2)

,WaterYear,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume,Flow_mean_pct_var,Flow_max_pct_var,Flow_min_pct_var,...,MeanQ3070,ExceedanceMean,ExceedanceMedian,ExceedanceMeanQ3070,rank,log_q,log_q_m2,log_q_m3,ret,exceed
33,1957,283.607969,803.689297,2333.913589,9912.101075,9628.493106,73.602299,112.831788,173.970469,31.792297,...,1004.311633,0.000,0.164,0.087,1,3.996166,0.245794,0.121859,94.0,0.010638
32,1956,290.521945,623.809012,1521.627983,9153.793636,8863.271691,47.986060,38.758695,153.010852,35.005214,...,663.367961,0.153,0.396,0.500,2,3.961601,0.212716,0.098107,47.0,0.021277


In [22]:
sum_m2=yearly['log_q_m2'].sum()
sum_m3=yearly['log_q_m3'].sum()

var=sum_m2/(n-1)
sd=var**0.5
skew=n*sum_m3/((n-1)*(n-2)*sd**3)
(var,sd,skew)

(0.05575894831673238, 0.23613332741638224, -0.47950674282666333)

In [23]:
skew_coef=pd.read_csv(input_data + 'skew_coefficients.csv').set_index('Cs')
skew_coef

,1.0101,2,5,10,25,50,100,200
Cs,,,,,,,,
3.0,-0.667,-0.396,0.420,1.180,2.278,3.152,4.051,4.970
2.9,-0.690,-0.390,0.440,1.195,2.277,3.134,4.013,4.904
2.8,-0.714,-0.384,0.460,1.210,2.275,3.114,3.973,4.847
2.7,-0.740,-0.376,0.479,1.224,2.272,3.093,3.932,4.783
2.6,-0.769,-0.368,0.499,1.238,2.267,3.071,3.889,4.718
...,...,...,...,...,...,...,...,...
-2.6,-3.899,0.368,0.696,0.747,0.764,0.768,0.769,0.769
-2.7,-3.932,0.376,0.681,0.724,0.738,0.740,0.740,0.741
-2.8,-3.973,0.384,0.666,0.702,0.712,0.714,0.714,0.714


In [24]:
skew_lower=np.floor(skew*10)/10
skew_upper=np.ceil(skew*10)/10
rets=pd.DataFrame([skew_coef.loc[skew_lower],skew_coef.loc[skew_upper]]).transpose()
rets['Slope']=rets[skew_lower]/rets[skew_upper]
rets['k']=rets[skew_upper]-((skew_upper-skew)*rets['Slope'])
rets['Q']=10**(av_log_q+(rets['k']*sd))
rets.index.name='Return'
rets=rets['Q']
rets

Return
1.0101      730.493833
2          3107.162500
5          4824.885158
10         5922.755947
25         7266.093448
50         8227.482380
100        9149.854371
200       10049.343083
Name: Q, dtype: float64

In [25]:
rets.to_csv(output_data + 'ngonye_floodreturns.csv')